In [96]:
from pyneuroml import pynml
import urllib.request, json 
#import requests
import os
import numpy as np
from neuroml import *
from neuroml.utils import component_factory, validate_neuroml2
from pyneuroml import pynml
from pyneuroml.lems import LEMSSimulation
from pyneuroml.lems import generate_lems_file_for_neuroml
import neuroml.writers as writers
import random
from pyneuroml.pynml import read_lems_file, read_neuroml2_file, write_lems_file, write_neuroml2_file


In [97]:
'''# Run the simulation using the default jNeuroML simulator
sim_id = "CortexNet"
lems_file_name = "LEMS_%s.xml" % sim_id
target_dir = "output_model"
lems_sim_filepath = os.path.join(target_dir, lems_file_name)
pynml.run_lems_with_jneuroml_neuron(lems_sim_filepath, 
                                    max_memory="2G", nogui=False, plot=False,
                                    verbose=True)'''

'# Run the simulation using the default jNeuroML simulator\nsim_id = "CortexNet"\nlems_file_name = "LEMS_%s.xml" % sim_id\ntarget_dir = "output_model"\nlems_sim_filepath = os.path.join(target_dir, lems_file_name)\npynml.run_lems_with_jneuroml_neuron(lems_sim_filepath, \n                                    max_memory="2G", nogui=False, plot=False,\n                                    verbose=True)'

In [98]:
'''https://pyneuroml.readthedocs.io/en/development/pyneuroml.io.html
pyneuroml.io.read_lems_file
pyneuroml.io.read_neuroml2_file
pyneuroml.io.write_lems_file
pyneuroml.io.write_neuroml2_file'''

# use the above functions to read LEMS and NeuroML files
cell_data_dir = "../NMC_model/NMC.NeuronML2/"
network_data_dir = "../NMC_model/"  # contains JSONs for network and connectivity data

files = os.listdir(cell_data_dir)
cell_files = [f for f in files if f.endswith("cell.nml")]  # .cell.nml -- contains arborization location details
net_files = [f for f in files if f.endswith("net.nml")]  # .net.nml -- contains location (0,0,0) in x-y-z space
current_files = [f for f in files if (f.endswith(".nml") and 'current' in f)]  # .current_clamp.nml -- contains current clamp details
print(cell_files[:5])
print(net_files[:5])
print(current_files[:5])

# validate the NeuroML files
'''for file in cell_files:
    print("Validating: " + file)
    validate_neuroml2(cell_data_dir + file)
for file in net_files:
    validate_neuroml2(cell_data_dir + file)
for file in current_files:
    validate_neuroml2(cell_data_dir + file)
'''

['bAC217_L23_BP_2004171a4c_0_0.cell.nml', 'bAC217_L23_BP_2f6bdee09f_0_0.cell.nml', 'bAC217_L23_BP_4652c9a7e1_0_0.cell.nml', 'bAC217_L23_BP_472b717f1f_0_0.cell.nml', 'bAC217_L23_BP_ca85cefc69_0_0.cell.nml']
['bAC217_L23_BP_2004171a4c.net.nml', 'bAC217_L23_BP_2004171a4c.stepcurrent3.net.nml', 'bAC217_L23_BP_2f6bdee09f.net.nml', 'bAC217_L23_BP_2f6bdee09f.stepcurrent3.net.nml', 'bAC217_L23_BP_4652c9a7e1.net.nml']
['bAC217_L23_BP_2004171a4c.stepcurrent3.net.nml', 'bAC217_L23_BP_2f6bdee09f.stepcurrent3.net.nml', 'bAC217_L23_BP_4652c9a7e1.stepcurrent3.net.nml', 'bAC217_L23_BP_472b717f1f.stepcurrent3.net.nml', 'bAC217_L23_BP_ca85cefc69.stepcurrent3.net.nml']


'for file in cell_files:\n    print("Validating: " + file)\n    validate_neuroml2(cell_data_dir + file)\nfor file in net_files:\n    validate_neuroml2(cell_data_dir + file)\nfor file in current_files:\n    validate_neuroml2(cell_data_dir + file)\n'

In [99]:
# read in the network and connection properties from jsons: circuit.json, layers.json
with open(network_data_dir + 'circuit.json') as f:
    circuit_data = json.load(f)  # layer thickness and neuron counts
with open(network_data_dir + 'layers.json') as f:
    layers_data = json.load(f)  # layer-specific me-type counts
with open(network_data_dir + 'pathways_anatomy.json') as f:
    pathways_anatomy_data = json.load(f)  # contains the synapse counts/distributions for each m_type->m_type connection
with open(network_data_dir + 'pathways_physiology.json') as f:
    pathways_physiology_data = json.load(f)  # contains the synapse properties for each m_type->m_type connection

# lock x-dimension None if xy cross-section is square, otherwise lock x-dim to the specified length
x_dim_length = None  # in microns  

# scale down factor for the network (no. of neurons)
# number of neurons is scaled down by this factor,
# neuron density, layer thickness, and connection probability are held constant
scale_down_factor_neuron = 100
for layer in circuit_data['No. of neurons per layer']:
    circuit_data['No. of neurons per layer'][layer] = int(circuit_data['No. of neurons per layer'][layer] / scale_down_factor_neuron)

# scale down factor for the synapses
# we only look at the first 1 / scale_down_factor_synapse synapses and
# preserve the connection probability. 
scale_down_factor_synapse = 1

# include only these layers (if None, include all layers)
filter_include_only_layers = ['L23', 'L4']

# how to select the me-type for each neuron in the network
# 'first' means the first me-type is selected
# 'all' means all me-types are included, weighted
me_type_selection_process = 'first' # 'first' or 'all'

# add to the circuit_data the location of the top edge of each layer
# order is L1, L23, L4, L5, L6
layer_thicknesses = circuit_data['Layer thicknesses']
layer_thicknesses['L23'] = layer_thicknesses['L2'] + layer_thicknesses['L3']
circuit_data['Neuron densities']['L23'] = (circuit_data['Neuron densities']['L2'] + circuit_data['Neuron densities']['L3']) / 2
circuit_data['Layer top edges'] = {'L1': 0}
layers = ['L1', 'L23', 'L4', 'L5', 'L6']
for i_l in range(1, len(layers)):
    prev_thickness = circuit_data['Layer top edges'][layers[i_l-1]] + layer_thicknesses[layers[i_l-1]]
    circuit_data['Layer top edges'][layers[i_l]] = prev_thickness

# add to each the layer-specific xy-area: (cell count / neuron density * 1000^3 )/ layer thickness
circuit_data['Layer areas'] = {}
circuit_data['x-length'] = {} # in microns
circuit_data['y-length'] = {} # in microns
for layer in layers:
    circuit_data['Layer areas'][layer] = (circuit_data['No. of neurons per layer'][layer] / circuit_data['Neuron densities'][layer] * 1000**3) / layer_thicknesses[layer]
    if x_dim_length is None:
        circuit_data['x-length'][layer] = np.sqrt(circuit_data['Layer areas'][layer])
        circuit_data['y-length'][layer] = circuit_data['x-length'][layer]
    else:
        circuit_data['x-length'][layer] = x_dim_length
        circuit_data['y-length'][layer] = circuit_data['Layer areas'][layer] / x_dim_length
    
print([layer + ": " + str(circuit_data['x-length'][layer])[:5] + " x "  + 
       str(circuit_data['y-length'][layer])[:5] + 
       ' um' for layer in layers])

['L1: 35.78 x 35.78 um', 'L23: 34.68 x 34.68 um', 'L4: 36.95 x 36.95 um', 'L5: 37.21 x 37.21 um', 'L6: 36.99 x 36.99 um']


In [100]:
nml_doc = component_factory("NeuroMLDocument", id="Cortex_Network")

### Create the network
net = nml_doc.add("Network", id="Cortex_Network", validate=False)
net.type="networkWithTemperature" 
net.temperature="34.0degC"

neuron_population_dict = {}  # neuron_population_dict[m_type][e_type] = population_object
microcircuit_volume = circuit_data["Microcircuit volume"]
microcircuit_thickness = sum(circuit_data["Layer thicknesses"].values())

total_cell_count = 0
for layer in layers_data:
    if layer not in filter_include_only_layers:
        continue
    layer_dict = layers_data[layer]
    e_type_counts = layer_dict['No. of neurons per electrical types']
    e_type_counts = {k: v // scale_down_factor_neuron for k, v in e_type_counts.items() if v // scale_down_factor_neuron > 0}
    m_type_counts = layer_dict['No. of neurons per morphological types'] 
    m_type_counts = {k: v // scale_down_factor_neuron for k, v in m_type_counts.items() if v // scale_down_factor_neuron > 0}

    # layer thickness
    layer_thickness = circuit_data["Layer thicknesses"][layer]
    layer_neuron_density = circuit_data["Neuron densities"][layer]
    layer_neuron_count = circuit_data["No. of neurons per layer"][layer]

    for m_type in m_type_counts:
        m_type_cells = [f for f in cell_files if m_type in f]
        m_count = m_type_counts[m_type]

        # for the e-types that match this m-type, build a relative probability distribution of these e-types based on their counts
        me_type_probs = {}
        me_type_counts = {}  # maps me-type to # neurons of that type
        e_count_sum = 0

        for e_type in e_type_counts:
            me_type_matches = [f for f in m_type_cells if e_type in f]
            for match in me_type_matches:
                me_type_probs[match] = e_type_counts[e_type] / len(me_type_matches)
            if len(me_type_matches) > 0:
                e_count_sum += e_type_counts[e_type]
        
        # normalize the probabilities
        for e_type in me_type_probs:
            me_type_probs[e_type] /= e_count_sum
        #print(me_type_probs)

        # probabilities should sum to 1
        if abs(1 - sum(me_type_probs.values())) > 0.0001 and len(me_type_probs.keys()) > 0:
            print("Error: probabilities do not sum to 1")
            print(sum(me_type_probs.values()))
            print(me_type_probs)

        # sample from the me-type distribution to get the number of cells of each type
        
        for i_morp in range(m_count):
            m_type_cells = list(me_type_probs.keys())
            m_type_probs = list(me_type_probs.values())
            if len(m_type_cells) == 0 or len(m_type_probs) == 0:
                continue
            me_type_sample = random.choices(m_type_cells, m_type_probs)[0]
            if me_type_sample not in me_type_counts:
                me_type_counts[me_type_sample] = 1
            else:
                me_type_counts[me_type_sample] += 1

        if me_type_selection_process == 'first':
            me_type_sample = list(me_type_counts.keys())[0]
            me_type_counts = {me_type_sample: sum(me_type_counts.values())}

        # create the neurons and place them in the network for this layer. Record in neuron_population_dict
        for me_type_sample in me_type_counts:
            print("Creating " + str(me_type_counts[me_type_sample]) + " cells of type " + me_type_sample)
            # read in the cell file
            cell_model = read_neuroml2_file(cell_data_dir + me_type_sample)

            # write the cell model to file
            write_neuroml2_file(cell_model, 'output_model/' + me_type_sample)

            # create the neuron population object and place them in the network for this layer
            size_pop1 = me_type_counts[me_type_sample]
            nml_doc.add("IncludeType", href=cell_data_dir + me_type_sample)

            print("cell_model.id", cell_model.id)
            me_cell_id = me_type_sample.replace(".cell.nml", "")

            population_obj = component_factory("Population", id=me_cell_id+"Cells", component=me_cell_id, size=size_pop1, type="populationList")
            net.add(population_obj)
            
            # Set optional color property. Note: used later when generating plots
            ##pop0.add("Property", tag="color", value="0 0 .8")
            
            # set the x,y,z location of the population, based on layer thickness (z), model x,y dimensions, and cell count of this population
            # place the population at a random x,y location within the layer,
            #  uniformly distributed according to layer_thickness, layer_neuron_density, and layer_neuron_count
            xy_avail_area = circuit_data['Layer areas'][layer]
            x_y_length = np.sqrt(xy_avail_area)
            z_start_bound = circuit_data['Layer top edges'][layer]
            z_end_bound = z_start_bound + layer_thickness
            for i_cell in range(size_pop1):
                x = random.uniform(0, x_y_length)
                y = random.uniform(0, x_y_length)
                z = random.uniform(z_start_bound, z_end_bound)
                population_obj.add("Instance", id=str(i_cell), location=component_factory("Location", x=x, y=y, z=z))

            # add to neuron_population_dict
            if m_type not in neuron_population_dict:
                neuron_population_dict[m_type] = {}
            neuron_population_dict[m_type][me_type_sample] = population_obj
            total_cell_count += size_pop1
print("Total cell count: " + str(total_cell_count))


pyNeuroML >>> pyneuroml.pynml - INFO - Loading NeuroML2 file: ../NMC_model/NMC.NeuronML2/bNAC219_L23_LBC_97118d2391_0_0.cell.nml


Creating 1 cells of type bNAC219_L23_LBC_97118d2391_0_0.cell.nml


pyNeuroML >>> 2025-02-05 18:33:21,382 - pyneuroml.pynml - INFO - Loading NeuroML2 file: ../NMC_model/NMC.NeuronML2/bNAC219_L23_LBC_97118d2391_0_0.cell.nml
pyNeuroML >>> pyneuroml.pynml - INFO - Executing: (java -Xmx400M  -jar  "C:\Users\jjudge3\AppData\Local\anaconda3\envs\in-silico-hVOS\Lib\site-packages\pyneuroml\lib\jNeuroML-0.11.0-jar-with-dependencies.jar" -validate "output_model/bNAC219_L23_LBC_97118d2391_0_0.cell.nml" ) in directory: .
pyNeuroML >>> 2025-02-05 18:33:21,609 - pyneuroml.pynml - INFO - Executing: (java -Xmx400M  -jar  "C:\Users\jjudge3\AppData\Local\anaconda3\envs\in-silico-hVOS\Lib\site-packages\pyneuroml\lib\jNeuroML-0.11.0-jar-with-dependencies.jar" -validate "output_model/bNAC219_L23_LBC_97118d2391_0_0.cell.nml" ) in directory: .
pyNeuroML >>> pyneuroml.pynml - CRITICAL - *** Problem running command: 
       Command 'java -Xmx400M  -jar  "C:\Users\jjudge3\AppData\Local\anaconda3\envs\in-silico-hVOS\Lib\site-packages\pyneuroml\lib\jNeuroML-0.11.0-jar-with-depend

ADD TO CODE: 1  jNeuroML v0.11.0
Validating: c:\Users\jjudge3\Desktop\scratch\in-silico-hVOS\output_model\bNAC219_L23_LBC_97118d2391_0_0.cell.nml
 Command: java -Xmx400M  -jar  "C:\Users\jjudge3\AppData\Local\anaconda3\envs\in-silico-hVOS\Lib\site-packages\pyneuroml\lib\jNeuroML-0.11.0-jar-with-dependencies.jar" -validate "output_model/bNAC219_L23_LBC_97118d2391_0_0.cell.nml" 
cell_model.id NeuroML2_file_exported_from_NEURON
Creating 1 cells of type bNAC219_L23_NBC_d27deeaf6a_0_0.cell.nml


pyNeuroML >>> pyneuroml.pynml - CRITICAL - *** Problem running command: 
       Command 'java -Xmx400M  -jar  "C:\Users\jjudge3\AppData\Local\anaconda3\envs\in-silico-hVOS\Lib\site-packages\pyneuroml\lib\jNeuroML-0.11.0-jar-with-dependencies.jar" -validate "output_model/bNAC219_L23_NBC_d27deeaf6a_0_0.cell.nml" ' returned non-zero exit status 1.
pyNeuroML >>> 2025-02-05 18:33:24,098 - pyneuroml.pynml - CRITICAL - *** Problem running command: 
       Command 'java -Xmx400M  -jar  "C:\Users\jjudge3\AppData\Local\anaconda3\envs\in-silico-hVOS\Lib\site-packages\pyneuroml\lib\jNeuroML-0.11.0-jar-with-dependencies.jar" -validate "output_model/bNAC219_L23_NBC_d27deeaf6a_0_0.cell.nml" ' returned non-zero exit status 1.
pyNeuroML >>> pyneuroml.pynml - CRITICAL -  jNeuroML >>   jNeuroML v0.11.0
 jNeuroML >>  Validating: c:\Users\jjudge3\Desktop\scratch\in-silico-hVOS\output_model\bNAC219_L23_NBC_d27deeaf6a_0_0.cell.nml
 jNeuroML >>  
pyNeuroML >>> 2025-02-05 18:33:24,098 - pyneuroml.pynml - CRITI

...

ADD TO CODE: 1  jNeuroML v0.11.0
Validating: c:\Users\jjudge3\Desktop\scratch\in-silico-hVOS\output_model\cADpyr230_L4_SS_a5ca6392b3_0_0.cell.nml
 Command: java -Xmx400M  -jar  "C:\Users\jjudge3\AppData\Local\anaconda3\envs\in-silico-hVOS\Lib\site-packages\pyneuroml\lib\jNeuroML-0.11.0-jar-with-dependencies.jar" -validate "output_model/cADpyr230_L4_SS_a5ca6392b3_0_0.cell.nml" 
cell_model.id NeuroML2_file_exported_from_NEURON
Creating 10 cells of type cADpyr230_L4_SP_c3f779f565_0_0.cell.nml


pyNeuroML >>> pyneuroml.pynml - CRITICAL - *** Problem running command: 
       Command 'java -Xmx400M  -jar  "C:\Users\jjudge3\AppData\Local\anaconda3\envs\in-silico-hVOS\Lib\site-packages\pyneuroml\lib\jNeuroML-0.11.0-jar-with-dependencies.jar" -validate "output_model/cADpyr230_L4_SP_c3f779f565_0_0.cell.nml" ' returned non-zero exit status 1.
pyNeuroML >>> 2025-02-05 18:33:31,510 - pyneuroml.pynml - CRITICAL - *** Problem running command: 
       Command 'java -Xmx400M  -jar  "C:\Users\jjudge3\AppData\Local\anaconda3\envs\in-silico-hVOS\Lib\site-packages\pyneuroml\lib\jNeuroML-0.11.0-jar-with-dependencies.jar" -validate "output_model/cADpyr230_L4_SP_c3f779f565_0_0.cell.nml" ' returned non-zero exit status 1.
pyNeuroML >>> pyneuroml.pynml - CRITICAL -  jNeuroML >>   jNeuroML v0.11.0
 jNeuroML >>  Validating: c:\Users\jjudge3\Desktop\scratch\in-silico-hVOS\output_model\cADpyr230_L4_SP_c3f779f565_0_0.cell.nml
 jNeuroML >>  
pyNeuroML >>> 2025-02-05 18:33:31,526 - pyneuroml.pynml - CRITI

ADD TO CODE: 1  jNeuroML v0.11.0
Validating: c:\Users\jjudge3\Desktop\scratch\in-silico-hVOS\output_model\cADpyr230_L4_SP_c3f779f565_0_0.cell.nml
 Command: java -Xmx400M  -jar  "C:\Users\jjudge3\AppData\Local\anaconda3\envs\in-silico-hVOS\Lib\site-packages\pyneuroml\lib\jNeuroML-0.11.0-jar-with-dependencies.jar" -validate "output_model/cADpyr230_L4_SP_c3f779f565_0_0.cell.nml" 
cell_model.id NeuroML2_file_exported_from_NEURON
Creating 26 cells of type cADpyr230_L4_PC_2d5807a37b_0_0.cell.nml


pyNeuroML >>> 2025-02-05 18:33:31,726 - pyneuroml.pynml - INFO - Executing: (java -Xmx400M  -jar  "C:\Users\jjudge3\AppData\Local\anaconda3\envs\in-silico-hVOS\Lib\site-packages\pyneuroml\lib\jNeuroML-0.11.0-jar-with-dependencies.jar" -validate "output_model/cADpyr230_L4_PC_2d5807a37b_0_0.cell.nml" ) in directory: .
pyNeuroML >>> pyneuroml.pynml - CRITICAL - *** Problem running command: 
       Command 'java -Xmx400M  -jar  "C:\Users\jjudge3\AppData\Local\anaconda3\envs\in-silico-hVOS\Lib\site-packages\pyneuroml\lib\jNeuroML-0.11.0-jar-with-dependencies.jar" -validate "output_model/cADpyr230_L4_PC_2d5807a37b_0_0.cell.nml" ' returned non-zero exit status 1.
pyNeuroML >>> 2025-02-05 18:33:32,590 - pyneuroml.pynml - CRITICAL - *** Problem running command: 
       Command 'java -Xmx400M  -jar  "C:\Users\jjudge3\AppData\Local\anaconda3\envs\in-silico-hVOS\Lib\site-packages\pyneuroml\lib\jNeuroML-0.11.0-jar-with-dependencies.jar" -validate "output_model/cADpyr230_L4_PC_2d5807a37b_0_0.cell.nm

ADD TO CODE: 1  jNeuroML v0.11.0
Validating: c:\Users\jjudge3\Desktop\scratch\in-silico-hVOS\output_model\cADpyr230_L4_PC_2d5807a37b_0_0.cell.nml
 Command: java -Xmx400M  -jar  "C:\Users\jjudge3\AppData\Local\anaconda3\envs\in-silico-hVOS\Lib\site-packages\pyneuroml\lib\jNeuroML-0.11.0-jar-with-dependencies.jar" -validate "output_model/cADpyr230_L4_PC_2d5807a37b_0_0.cell.nml" 
cell_model.id NeuroML2_file_exported_from_NEURON
Total cell count: 79


In [101]:
total_connections = 0
syn_count = 0
for m1m2_key in pathways_anatomy_data:
    syn_anatomy = pathways_anatomy_data[m1m2_key]
    syn_physiology = pathways_physiology_data[m1m2_key]

    m1, m2 = m1m2_key.split(":")
    connection_prob = syn_anatomy["connection_probability"]
    decay_mean = syn_physiology["decay_mean"]

    # simple mono-exponential synapse
    # <Property name="weight" dimension="none" defaultValue="1"/>

    # <Parameter name="tauDecay" dimension="time" description="Time course of decay"/>
    # <Parameter name="gbase" dimension="conductance" description="Baseline conductance, generally the maximum conductance following a single spike"/>
    # <Parameter name="erev" dimension="voltage" description="Reversal potential of the synapse"/>
    # Create connections and inputs
    proj_count = 0
    syn0 = nml_doc.add(
        "ExpOneSynapse", id="Syn" + str(syn_count), gbase="65nS", erev="0mV", tau_decay= str(decay_mean) + "ms"
    )
    syn_count += 1
    
    #print("me-types:")
    #print(neuron_population_dict[m1])
    #print(neuron_population_dict[m2])
    # for every pair of me-types, create a fraction of the synapses
    if m1 not in neuron_population_dict or m2 not in neuron_population_dict:
        continue
    for me1 in neuron_population_dict[m1]:
        me_pop1 = neuron_population_dict[m1][me1]
        for me2 in neuron_population_dict[m2]:
            me_pop2 = neuron_population_dict[m2][me2]

            pre_pop_id = me1.replace(".cell.nml", "")
            post_pop_id = me2.replace(".cell.nml", "")

            

            projection = Projection(
                                id="Proj_",
                                presynaptic_population=pre_pop_id,
                                postsynaptic_population=post_pop_id,
                                synapse=syn0.id,
                            )

            net.projections.append(projection)
            for i in range(me_pop1.size):
                # scale down the number of synapses
                num_connections_max = max(10, me_pop2.size // scale_down_factor_synapse)
                num_connections_max = min(num_connections_max, me_pop2.size) 
                for j in range(num_connections_max): 
                    if random.random() <= connection_prob: # probablistic connection...
                        connection = ConnectionWD(
                            id=proj_count,
                            pre_cell_id="../%sCells/%i/%s" % (pre_pop_id, i, pre_pop_id),
                            post_cell_id="../%sCells/%i/%s" % (post_pop_id, j, post_pop_id),
                            weight=random.random(),
                            delay='0ms'
                        )
                        projection.add(connection)
                        proj_count += 1
            print("\tAdded %i connections from %s to %s" % (proj_count, me1, me2))
            total_connections += proj_count
print("Total connections: %i" % total_connections + "\n\n")



	Added 1 connections from bNAC219_L23_NBC_d27deeaf6a_0_0.cell.nml to cNAC187_L4_MC_2eb83e1eed_0_0.cell.nml
	Added 884 connections from cADpyr229_L23_PC_863902f300_0_0.cell.nml to cADpyr230_L4_PC_2d5807a37b_0_0.cell.nml
	Added 1 connections from cACint209_L4_LBC_66ec8fed8f_0_0.cell.nml to bNAC219_L23_LBC_97118d2391_0_0.cell.nml
	Added 10 connections from cACint209_L23_MC_940ffcbb27_0_0.cell.nml to cADpyr230_L4_SP_c3f779f565_0_0.cell.nml
	Added 4 connections from cACint209_L23_MC_940ffcbb27_0_0.cell.nml to cADpyr230_L4_SS_a5ca6392b3_0_0.cell.nml
	Added 1 connections from cACint209_L23_MC_940ffcbb27_0_0.cell.nml to cACint209_L23_MC_940ffcbb27_0_0.cell.nml
	Added 1 connections from cACint209_L23_MC_940ffcbb27_0_0.cell.nml to cACint209_L4_LBC_66ec8fed8f_0_0.cell.nml
	Added 34 connections from cADpyr229_L23_PC_863902f300_0_0.cell.nml to cACint209_L4_LBC_66ec8fed8f_0_0.cell.nml
	Added 1 connections from cNAC187_L4_MC_2eb83e1eed_0_0.cell.nml to cACint209_L4_LBC_66ec8fed8f_0_0.cell.nml
	Added 1

In [102]:
# stimulus: L4 spiny stellate neurons (simulating thalamic input) as a pulse generator at 20ms, duration 0.2ms, amplitude on 10^-1 nA scale
for me_pop_key in neuron_population_dict["L4_SS"]:
    me_pop = neuron_population_dict["L4_SS"][me_pop_key]
    for i in range(0, me_pop.size):
        # pulse generator as explicit stimulus
        pg = nml_doc.add(
            "PulseGenerator",
            id="pg_exc_%i" % i,
            delay="20ms",
            duration="0.2ms",
            amplitude="%f nA" % (0.3 + 0.1 * random.random()),
        )
        pop_id = me_pop_key.replace(".cell.nml", "")
        exp_input = net.add(
            "ExplicitInput", target="../%sCells/%i/%s" % (pop_id, i, pop_id), input=pg.id, validate=False
        )

print(nml_doc.summary())

nml_net_file = 'Cortex_Network.net.nml'
writers.NeuroMLWriter.write(nml_doc, nml_net_file)

print("Written network file to: " + nml_net_file)


*******************************************************
* NeuroMLDocument: Cortex_Network
*
*  ExpOneSynapse: ['Syn0', 'Syn1', 'Syn10', 'Syn100', 'Syn1000', 'Syn1001', 'Syn1002', 'Syn1003', 'Syn1004', 'Syn1005', 'Syn1006', 'Syn1007', 'Syn1008', 'Syn1009', 'Syn101', 'Syn1010', 'Syn1011', 'Syn1012', 'Syn1013', 'Syn1014', 'Syn1015', 'Syn1016', 'Syn1017', 'Syn1018', 'Syn1019', 'Syn102', 'Syn1020', 'Syn1021', 'Syn1022', 'Syn1023', 'Syn1024', 'Syn1025', 'Syn1026', 'Syn1027', 'Syn1028', 'Syn1029', 'Syn103', 'Syn1030', 'Syn1031', 'Syn1032', 'Syn1033', 'Syn1034', 'Syn1035', 'Syn1036', 'Syn1037', 'Syn1038', 'Syn1039', 'Syn104', 'Syn1040', 'Syn1041', 'Syn1042', 'Syn1043', 'Syn1044', 'Syn1045', 'Syn1046', 'Syn1047', 'Syn1048', 'Syn1049', 'Syn105', 'Syn1050', 'Syn1051', 'Syn1052', 'Syn1053', 'Syn1054', 'Syn1055', 'Syn1056', 'Syn1057', 'Syn1058', 'Syn1059', 'Syn106', 'Syn1060', 'Syn1061', 'Syn1062', 'Syn1063', 'Syn1064', 'Syn1065', 'Syn1066', 'Syn1067', 'Syn1068', 'Syn1069', 'Syn107', 'Syn1070', 'Sy

In [103]:
# next, generate a LEMS file to run the simulation, with helper method
# https://docs.neuroml.org/Userdocs/LEMSSimulation.html
# https://github.com/NeuroML/pyNeuroML/blob/master/pyneuroml/lems/__init__.py#L19 

sim_id = "Cortex_Network"
target = "Cortex_Network"
duration = 200
dt = 0.025
lems_file_name = "LEMS_%s.xml" % sim_id
target_dir = "output_model"


generate_lems_file_for_neuroml(
    sim_id,
    nml_net_file,
    target,
    duration,
    dt,
    lems_file_name,
    target_dir,
    include_extra_files=[],
    gen_plots_for_all_v=True,
    plot_all_segments=True,
    gen_plots_for_quantities={},  # Dict with displays vs lists of quantity paths
    gen_plots_for_only_populations=[],  # List of populations, all pops if = []
    gen_saves_for_all_v=True,
    save_all_segments=True,
    gen_saves_for_only_populations=[],  # List of populations, all pops if = []
    gen_saves_for_quantities={}, # Dict with file names vs lists of quantity paths
    gen_spike_saves_for_all_somas=True,
    report_file_name="report.txt",
    copy_neuroml=True,
    verbose=True,
)

# output file name
print("Written LEMS file to: " + target_dir + "/" + lems_file_name)

pyNeuroML >>> pyneuroml.lems - INFO - Creating LEMS file at: output_model/LEMS_CortexNet.xml for NeuroML 2 file: Cortex_Network.net.nml (copy: True)
pyNeuroML >>> 2025-02-05 18:33:49,753 - pyneuroml.lems - INFO - Creating LEMS file at: output_model/LEMS_CortexNet.xml for NeuroML 2 file: Cortex_Network.net.nml (copy: True)
pyNeuroML >>> pyneuroml.pynml - INFO - Loading NeuroML2 file: Cortex_Network.net.nml
pyNeuroML >>> 2025-02-05 18:33:49,769 - pyneuroml.pynml - INFO - Loading NeuroML2 file: Cortex_Network.net.nml
pyNeuroML >>> pyneuroml.pynml - INFO - Including included files (included already: ['C:\\Users\\jjudge3\\Desktop\\scratch\\NMC_model\\NMC.NeuronML2\\K_Tst.channel.nml', 'C:\\Users\\jjudge3\\Desktop\\scratch\\NMC_model\\NMC.NeuronML2\\SKv3_1.channel.nml', 'C:\\Users\\jjudge3\\Desktop\\scratch\\NMC_model\\NMC.NeuronML2\\Nap_Et2.channel.nml', 'C:\\Users\\jjudge3\\Desktop\\scratch\\NMC_model\\NMC.NeuronML2\\KdShu2007.channel.nml', 'C:\\Users\\jjudge3\\Desktop\\scratch\\NMC_model\

Written LEMS file to: output_model/LEMS_CortexNet.xml


In [104]:
net_nml_filepath = target_dir + "/" + nml_net_file
lems_sim_filepath = target_dir + "/" + lems_file_name
print(net_nml_filepath)

def replace_in_file(file_path, old_str, new_str):
    with open(file_path, 'r') as file:
        filedata = file.read()
    filedata = filedata.replace(old_str, new_str)
    os.remove(file_path)
    with open(file_path, 'w') as file:
        file.write(filedata)

# replace all "../NMC_model/NMC.NeuronML2/" with "output_model/" in the LEMS file
replace_in_file(lems_sim_filepath, "../NMC_model/NMC.NeuronML2/", "output_model/")
replace_in_file(net_nml_filepath, "../NMC_model/NMC.NeuronML2/", "output_model/")

# validate the LEMS file
read_lems_file(lems_sim_filepath)

output_model/Cortex_Network.net.nml


In [105]:
# for each cell file, read in lines. If the line contains "biophysicalProperties", and
# the next line is "<membraneProperties>", and no following lines have 'spikeThresh' then add the following line:
# add a spike threshold as follows:
# <spikeThresh value="-20.0mV"/>
# to a new line. Write the new file back to the output_model directory.
for file in os.listdir('output_model/'):
    if file.endswith(".cell.nml"):
        print("Adding spike threshold to: " + 'output_model/' + file)
        with open('output_model/' + file, 'r') as f:
            lines = f.readlines()
        # remove file
        os.remove('output_model/' + file)
        with open('output_model/' + file, 'w') as f:
            for i, line in enumerate(lines):
                cell_id = file.replace(".cell.nml", "")
                f.write(line.replace("NeuroML2_file_exported_from_NEURON", cell_id))
                if '<membraneProperties>' in line and i>0 and 'biophysicalProperties' in lines[i-1]:
                    if 'spikeThresh' not in lines[i+1]:
                        f.write('    <spikeThresh value="-20.0mV"/>\n')

        # remove same file in cell_data_dir
        try:
            os.remove(cell_data_dir + file)
        except FileNotFoundError:
            pass
        # copy the new file to cell_data_dir
        os.system('cp output_model/' + file + ' ' + cell_data_dir + file)

Adding spike threshold to: output_model/bNAC219_L23_LBC_91044b50b3_0_0.cell.nml
Adding spike threshold to: output_model/bNAC219_L23_LBC_936c35a9e1_0_0.cell.nml
Adding spike threshold to: output_model/bNAC219_L23_LBC_97118d2391_0_0.cell.nml
Adding spike threshold to: output_model/bNAC219_L23_LBC_b35128281c_0_0.cell.nml
Adding spike threshold to: output_model/bNAC219_L23_LBC_fe2122c75c_0_0.cell.nml
Adding spike threshold to: output_model/bNAC219_L23_MC_31f333a601_0_0.cell.nml
Adding spike threshold to: output_model/bNAC219_L23_MC_44b474bb8f_0_0.cell.nml
Adding spike threshold to: output_model/bNAC219_L23_MC_940ffcbb27_0_0.cell.nml
Adding spike threshold to: output_model/bNAC219_L23_NBC_01d2e3caa6_0_0.cell.nml
Adding spike threshold to: output_model/bNAC219_L23_NBC_6561f8d865_0_0.cell.nml
Adding spike threshold to: output_model/bNAC219_L23_NBC_68416e1f93_0_0.cell.nml
Adding spike threshold to: output_model/bNAC219_L23_NBC_d27deeaf6a_0_0.cell.nml
Adding spike threshold to: output_model/bNA

In [106]:
# first validate all the cell files
for file in os.listdir('output_model/'):
    if file.endswith(".cell.nml"):
        print("Validating: " + 'output_model/' + file)
        validate_neuroml2('output_model/' + file)
        # and validate in cell_data_dir
        validate_neuroml2(cell_data_dir + file)


Validating: output_model/bNAC219_L23_LBC_91044b50b3_0_0.cell.nml
It's valid!
It's valid!
Validating: output_model/bNAC219_L23_LBC_936c35a9e1_0_0.cell.nml
It's valid!
It's valid!
Validating: output_model/bNAC219_L23_LBC_97118d2391_0_0.cell.nml
It's valid!
It's valid!
Validating: output_model/bNAC219_L23_LBC_b35128281c_0_0.cell.nml
It's valid!
It's valid!
Validating: output_model/bNAC219_L23_LBC_fe2122c75c_0_0.cell.nml
It's valid!
It's valid!
Validating: output_model/bNAC219_L23_MC_31f333a601_0_0.cell.nml
It's valid!
It's valid!
Validating: output_model/bNAC219_L23_MC_44b474bb8f_0_0.cell.nml
It's valid!
It's valid!
Validating: output_model/bNAC219_L23_MC_940ffcbb27_0_0.cell.nml
It's valid!
It's valid!
Validating: output_model/bNAC219_L23_NBC_01d2e3caa6_0_0.cell.nml
It's valid!
It's valid!
Validating: output_model/bNAC219_L23_NBC_6561f8d865_0_0.cell.nml
It's valid!
It's valid!
Validating: output_model/bNAC219_L23_NBC_68416e1f93_0_0.cell.nml
It's valid!
It's valid!
Validating: output_model

In [107]:
# validate nml file
validate_neuroml2(target_dir + "/Cortex_Network.net.nml" )

It's valid!


In [108]:
# Run the simulation using the default jNeuroML simulator
pynml.run_lems_with_jneuroml_neuron(lems_sim_filepath, 
                                    max_memory="2G", nogui=False, plot=False,
                                    verbose=True)

pyNeuroML >>> pyneuroml.pynml - INFO - Loading LEMS file: output_model/LEMS_CortexNet.xml and running with jNeuroML_NEURON
pyNeuroML >>> 2025-02-05 18:34:50,853 - pyneuroml.pynml - INFO - Loading LEMS file: output_model/LEMS_CortexNet.xml and running with jNeuroML_NEURON
pyNeuroML >>> pyneuroml.pynml - INFO - Executing: (java -Xmx2G  -jar  "C:\Users\jjudge3\AppData\Local\anaconda3\envs\in-silico-hVOS\Lib\site-packages\pyneuroml\lib\jNeuroML-0.11.0-jar-with-dependencies.jar"  "output_model/LEMS_CortexNet.xml"  -neuron -run -compile -I '') in directory: .
pyNeuroML >>> 2025-02-05 18:34:50,855 - pyneuroml.pynml - INFO - Executing: (java -Xmx2G  -jar  "C:\Users\jjudge3\AppData\Local\anaconda3\envs\in-silico-hVOS\Lib\site-packages\pyneuroml\lib\jNeuroML-0.11.0-jar-with-dependencies.jar"  "output_model/LEMS_CortexNet.xml"  -neuron -run -compile -I '') in directory: .
pyNeuroML >>> pyneuroml.pynml - CRITICAL - *** Problem running command: 
       Command 'java -Xmx2G  -jar  "C:\Users\jjudge3\

ADD TO CODE: 1  jNeuroML v0.11.0
(INFO) Reading from: C:\Users\jjudge3\Desktop\scratch\in-silico-hVOS\output_model\LEMS_CortexNet.xml
(WARNING) Unrecognized attribute XMLAttribute(description, Name of the property)  Name of the property at: org.lemsml.jlems.core.reader.LemsFactory.buildText(LemsFactory.java:950)
(LOG)  .......  last warning repeated 3 times
(LOG)  .......  last warning repeated 10 times
(LOG)  .......  last warning repeated 30 times
 Command: java -Xmx2G  -jar  "C:\Users\jjudge3\AppData\Local\anaconda3\envs\in-silico-hVOS\Lib\site-packages\pyneuroml\lib\jNeuroML-0.11.0-jar-with-dependencies.jar"  "output_model/LEMS_CortexNet.xml"  -neuron -run -compile -I -I


SystemExit: 1

c:\Users\jjudge3\AppData\Local\anaconda3\envs\in-silico-hVOS\lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [25]:
%tb

SystemExit: 1